In [4]:
#Object Relational Mapper
#A library that offers you an object interface to the data in your database
#Layer of abstraction over the database: It means you don't have to write SQL code 
#or perform transactions with the database ORMs
#Independently you're using PostgreSQL, MySQL, SQLite the code doesn't changes.
#Object-Oriented Interface to the Data, based on classes that you define in your code
#It permits to leveraging the benefits of OOP (Object Oriented Programming), 
#to reach those classes with additional properties and methods that can make it easier to work with your data 

#Alembic
#Allows you to replicate in the database the schemas of the data that you have 
#defined in your code with sql alchemy models
#The schemas in our database are an accurate reflection of the models in our code 

In [5]:
pip install alembic sqlalchemy

In [3]:
from datetime import datetime
from sqlalchemy import Column, Integer, String, CHAR, DateTime, create_engine, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
Base = declarative_base()
engine = create_engine('sqlite:///models.db', pool_size=10, max_overflow=20)
# Drop existing tables if they exist
Base.metadata.drop_all(engine)

In [4]:
class UserModel(Base):
    __tablename__ = "users"

    id = Column("id", Integer, primary_key = True)
    first_name = Column("firstname", String, nullable = False)
    last_name = Column("lastname", String, nullable = False)
    gender = Column("gender", CHAR)
    birth = Column("birth", DateTime)
    created= Column("created", DateTime, default=datetime.utcnow)
    things = relationship("Thing", back_populates="user", passive_deletes=True)
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    def __init__(self, first, last, gender, birth):
        #self.id = id
        self.first_name = first
        self.last_name = last
        self.gender = gender
        self.birth = birth
        #self.created = created

    def __repr__(self):
        return(
            f"UserModel(id={self.id}, first_name={self.first_name},"
            f"last_name={self.last_name}, birth={self.birth},"
            f"created={self.created}"
        )

In [5]:
class Thing(Base):
    __tablename__ = "things"
    __table_args__ = {"extend_existing": True}
    
    tid = Column("tid", Integer, primary_key=True)
    description = Column("description", String)
    owner = Column(Integer, ForeignKey("users.id"))
    
    user = relationship("UserModel", back_populates="things")
    def __init__(self, description, owner):
        #self.tid = tid,
        self.description = description
        self.owner = owner
        
    def __repr__(self):
        return f"({self.tid}) {self.description} owned by {self.owner}"


In [6]:


Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
users = [
    UserModel(first="Giacomo", last="Bonadiman", gender='m', birth=datetime(1996, 8, 25)),
    UserModel(first="Veronica", last="Bonadiman", gender='f', birth=datetime(1985, 9, 4)),
    UserModel(first="Paola", last="Valle", gender='f', birth=datetime(1957, 11, 9))
]

# Esempio di inserimento dati
for user in users:
    session.add(user)

session.commit()


In [8]:
# Esempi di query
users = session.query(UserModel).all()
results = session.query(UserModel).filter(UserModel.last_name=="Valle")
char_like = session.query(UserModel).filter(UserModel.first_name.like("%i%"))
part = session.query(UserModel).filter(UserModel.first_name.in_(["Veronica", "Giacomo"]))

In [9]:
things = [
    Thing(description="CAR", owner=users[0].full_name),
    Thing(description="LAPTOP", owner=users[0].full_name),
    Thing(description="PS5", owner=users[1].full_name),
    Thing(description="TOOL", owner=users[2].full_name),
    Thing(description="BOOK", owner=users[2].full_name)
]

# Esempio di inserimento dati
for thing in things:
    session.add(thing)

session.commit()

In [10]:
# Query things
thing_records = session.query(Thing).all()
results = session.query(Thing, UserModel).filter(Thing.owner == UserModel.id).filter(UserModel.first_name == "Giacomo").all()

for thing in thing_records:
    print(thing)

(1) CAR owned by 1
(2) LAPTOP owned by 1
(3) PS5 owned by 2
(4) TOOL owned by 3
(5) BOOK owned by 3
(6) CAR owned by Giacomo Bonadiman
(7) LAPTOP owned by 1
(8) PS5 owned by 2
(9) TOOL owned by 3
(10) BOOK owned by 3
(11) CAR owned by Giacomo Bonadiman
(12) LAPTOP owned by Giacomo Bonadiman
(13) PS5 owned by Veronica Bonadiman
(14) TOOL owned by Paola Valle
(15) BOOK owned by Paola Valle
(16) CAR owned by Giacomo Bonadiman
(17) LAPTOP owned by Giacomo Bonadiman
(18) PS5 owned by Veronica Bonadiman
(19) TOOL owned by Paola Valle
(20) BOOK owned by Paola Valle


In [14]:
with Session() as session:
    user_records = session.query(UserModel).all()
    for user in user_records:
        print(user.full_name)
        


Giacomo Bonadiman
Veronica Bonadiman
Paola Valle
Giacomo Bonadiman
Veronica Bonadiman
Paola Valle


In [15]:
with Session() as session:
    thing_records = session.query(Thing).all()
    for thing in thing_records:
        print(thing)

(1) CAR owned by 1
(2) LAPTOP owned by 1
(3) PS5 owned by 2
(4) TOOL owned by 3
(5) BOOK owned by 3
(6) CAR owned by Giacomo Bonadiman
(7) LAPTOP owned by 1
(8) PS5 owned by 2
(9) TOOL owned by 3
(10) BOOK owned by 3
(11) CAR owned by Giacomo Bonadiman
(12) LAPTOP owned by Giacomo Bonadiman
(13) PS5 owned by Veronica Bonadiman
(14) TOOL owned by Paola Valle
(15) BOOK owned by Paola Valle
(16) CAR owned by Giacomo Bonadiman
(17) LAPTOP owned by Giacomo Bonadiman
(18) PS5 owned by Veronica Bonadiman
(19) TOOL owned by Paola Valle
(20) BOOK owned by Paola Valle
